# Доверительные интервалы. Тест первой недели 

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

In [1]:
import pandas as pd
import scipy as sp
import numpy as np
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

In [2]:
df = pd.read_csv('data/water.txt', sep = '\t')
df.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
location     61 non-null object
town         61 non-null object
mortality    61 non-null int64
hardness     61 non-null int64
dtypes: int64(2), object(2)
memory usage: 2.0+ KB


## Средняя годовая смертность
Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Будьте осторожны при использовании метода std()! Дело в том, что у объекта numpy он по умолчанию вычисляется как $\frac{1}{n}∑_{i=1}^n (X_i−\bar{X}^2)$, а у объекта pandas — как $\frac{1}{n−1}∑_{i=1}^n (X_i−\bar{X}^2)$.

Нас интересует только второй вариант, несмещённая оценка стандартного отклонения.

Чтобы не думать всё время о том, правильно ли вычисляется в вашем случае std(), можно всегда использовать std(ddof=1) (ddof — difference in degrees of freedom), тогда нормировка всегда будет на $n-1$.

In [4]:
n = df.shape[0]

mean = df['mortality'].mean()
std = df['mortality'].std(ddof = 1)

print("mean 95% confidence interval", _tconfint_generic(mean, std/np.sqrt(n), n - 1, 0.05, 'two-sided'))

mean 95% confidence interval (1476.0833413552848, 1572.2117406119285)


In [5]:
def compare_cities_stats(row):
    locations = df['location'].unique()
    for location in locations:
        df_location = df[df['location'] == location]
        mean = df_location[row].mean()
        std = df_location[row].std(ddof = 1)
        n = df_location.shape[0]
        print("mean 95% confidence interval for " + location + " cities", _tconfint_generic(mean, std/np.sqrt(n), n - 1, 0.05, 'two-sided'))

## Средняя годовая смертность по городам
На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод? 
- Интервалы не пересекаются; видимо, средняя годовая смертность на севере и на юге существенно разная.

In [6]:
compare_cities_stats('mortality')

mean 95% confidence interval for South cities (1320.1517462936238, 1433.463638321761)
mean 95% confidence interval for North cities (1586.5605251961385, 1680.6394748038613)


## Средняя жесткость воды
Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?
- Интервалы не пересекаются - средние разные.

In [7]:
compare_cities_stats('hardness')

mean 95% confidence interval for South cities (53.467198692036106, 86.07126284642544)
mean 95% confidence interval for North cities (21.42248728572426, 39.37751271427574)


## Объём выборки

Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией $\sigma^2$:

$$\bar{X}_n±z_{1−\frac{α}{2}} \frac{σ}{\sqrt{n}}$$

При $\sigma=1$ какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью $\pm0.1$?

### Ответ
Путём хитрых манипуляций, можно получить следующее выражение:
$$n = \frac{z_{1−\frac{α}{2}}^2 σ^2}{ширина^2}$$

In [8]:
z = sp.stats.norm.ppf(1 - 0.05 / 2)  
sigma = 1
width = 0.1

n = np.ceil((z**2 * sigma**2) / width**2)

print('Объём выборки при заданных условиях', n)

Объём выборки при заданных условиях 385.0


## Результат
Был закреплен материал из лекций. Когда нам неизвестна теоретическая дисперсия, следует использовать критерий Стьюдента.